# plansquad intake prototyping

In [1]:
%%capture --no-stderr
%pip install -U langgraph langsmith
%pip install -U tavily-python
%pip install -U langchain_community
%pip install -U langchain-groq


## Set up the environment

In [2]:
import getpass
import os


def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

_set_env("GROQ_API_KEY")
_set_env("TAVILY_API_KEY")

In [3]:
_set_env("LANGSMITH_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "PlanSquad Intake Test"

In [4]:
# Make sure we're using the decrypted secrets
assert(os.environ["LANGSMITH_API_KEY"].startswith("lsv2_"))

## Load a chat engine

In [5]:
from langchain_groq import ChatGroq

model = ChatGroq(model="llama-3.1-8b-instant")

## Build a prompt template

In [6]:
from langchain_core.prompts import ChatPromptTemplate

# From https://smith.langchain.com/hub/mcdiddy/knowledgebasejsoniterator

prompt = ChatPromptTemplate.from_messages([
  ("system", '''
  You are a helpful, skilled analyst that assists people in creating a
  detailed plan. You will need to:
  1. Determine what type of plan they're creating with you;
  2. Extract the specific information needed for the plan;
  3. Ask clarifying questions or suggest a few clarifications of your own;
  4. Iterate until you can generate the plan;
  5. Always defer to the person you're helping as the ultimate authority.
'''),
  ("human", "{question}"),
])


## Execute

In [7]:
messages  = prompt.invoke({"question": '''
I want to create training for a group of salespeople on how to sell my new software. Help me make a plan.
'''}).messages

model.invoke(messages)

AIMessage(content="To create an effective training plan for your sales team on selling your new software, we'll need to break it down into several key components. Let's start with the basics.\n\nCan you tell me a bit more about your new software? For example:\n\n* What problem does it solve for customers?\n* What are its key features and benefits?\n* Who is the target audience for this software?\n* What are the primary pain points or challenges that your sales team will be addressing with this software?\n\nAlso, what are your goals for this training? Are you looking to:\n\n* Introduce the software to new sales team members?\n* Refresh existing sales team members on the software?\n* Improve sales performance and close rates with the software?\n* Develop a consistent sales message and pitch?\n\nThis will help us determine the scope and focus of the training plan.", response_metadata={'token_usage': {'completion_tokens': 169, 'prompt_tokens': 158, 'total_tokens': 327, 'completion_time': 0